In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def htmlden_doktorlari_al(html):
  soup = BeautifulSoup(html, 'html.parser')
  az_content = soup.find('div', class_='az-content')
  az_main_wrappers = az_content.find_all('div', class_='az-main-wrapper')
  doktorlar = []
  for az_main_wrapper in az_main_wrappers:
      verified = az_main_wrapper.find('div', class_='verified')
      dogrulanmis_profil = False
      if verified is not None:
          dogrulanmis_profil = verified.text
      resim = az_main_wrapper.find('img')
      resim_linki = resim['src']
      cinsiyet = resim['data-gender']
      profil = az_main_wrapper.find('a')
      # <a href="https://www.doktorsitesi.com/ahmet-dincer/psikoloji/istanbul"> <span>Psk.</span> Ahmet Dinçer </a>        
      profil_linki = profil['href']
      konum = profil_linki.split('/')[-1]
      uzmanlik_alani = profil_linki.split('/')[-2]
      # unvanı <span>Psk.</span> olarak alır
      unvan = profil.find('span').text
      # ismi buradan alıp ve temizler \nPsk.\n                                Arzu Kantarcıoğlu\n
      isim = profil.text.split('\n')[2].strip()

      doktorlar.append({
          'resim_linki': resim_linki,
          'unvan': unvan,
          'isim': isim,
          'dogrulanmis_profil': dogrulanmis_profil,
          'profil_linki': profil_linki,
          'cinsiyet': cinsiyet,
          'konum': konum,
          'uzmanlik_alani': uzmanlik_alani
      })
  return doktorlar

In [ ]:
tum_doktorlar = []
for i in range(1, 999):
    response = requests.get("https://www.doktorsitesi.com/tumuzmanlar?sayfa=" + str(i))
    doktorlar = htmlden_doktorlari_al(response.text)
    tum_doktorlar.extend(doktorlar)

doktorlar_data_frame = pd.DataFrame(
    columns=['resim_linki', 'unvan', 'isim', 'dogrulanmis_profil', 'profil_linki', 'cinsiyet', 'konum', 'uzmanlik_alani'],
    data=tum_doktorlar
)

doktorlar_data_frame.to_csv('tum_uzmanlar.csv', index=False)
doktorlar_data_frame.head()